<a href="https://colab.research.google.com/github/Jan-Agatz/loss-landscape/blob/no_mpi/MML_Paper_04_Implementations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

Folder with data: https://drive.google.com/drive/u/0/folders/19-zhmQXrtExX5Ddhj3s_BePBX7ELy9XA

## Mounting Google Drive to access and save files.

In [2]:
# USER-SPECIFIED: Input your name
NAME = "JHA"

# USER-SPECIFIED: Input your name
if NAME == "JHA":
  # USER-SPECIFIED: Input the mounting path for your Google Drive
  MOUNT_PATH = "/content/drive"

  # USER-SPECIFIED: Input the path to the folder containing the code and models
  CODE_PATH  = "/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code"

In [3]:
from google.colab import drive
drive.mount(MOUNT_PATH)

Mounted at /content/drive


In [4]:
%cd $CODE_PATH
%ls

/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code
 cifar10/            LICENSE                 plot_trajectory.py
 cifar100/           MNIST/                  projection.py
 data/               model_loader.py         __pycache__/
 dataloader.py       mpi4pytorch.py          README.md
 evaluation.py       net_plotter.py          scheduler.py
 FashionMNIST/       pbs_job_launch.py       script/
 h52vtp.py           plot_1D.py              test.py
 h5_util.py          plot_2D.py              trained_nets/
 hess_vec_prod.py    plot_hessian_eigen.py   _weights.h5
 launch_cluster.py   plot_surface.py        '_weights.h5_[-1.0,1.0,101]x[-1.0,1.0,101].h5'


## Installing libraries (with the newest versions)

In [5]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746511 sha256=ea1749f8845b7b9ba34cc63d25db47c0292841008c19be3b3ed186ee5a48036b
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [7]:
# Checks whether the necessary libraries have already been installed
try:
  import os
  import time
  import torch
  import torchvision
  import numpy as np
  import mpi4py
  import h5py
  import matplotlib
  import scipy
except:
  print("Please install the necessary libraries")

In [8]:
# Check whether CUDA is available in the current environment and adjusts the
# bash commands accordingly
CUDA_AVAILABLE = torch.cuda.is_available()

if CUDA_AVAILABLE:
  CUDA_FLAG = "--cuda"
else:
  CUDA_FLAG = ""

print(CUDA_AVAILABLE)

True


## Training models on different datasets

In [8]:
TRAINING_FINISHED = True

In [9]:
if not TRAINING_FINISHED:
  !python FashionMNIST/main.py --model resnet56 --save_epoch 1

In [10]:
if not TRAINING_FINISHED:
  !python MNIST/main.py --model resnet56 --save_epoch 1

In [11]:
# Currently does not work
# The model never leaves the 95% error rate
if not TRAINING_FINISHED:
  !python cifar100/main.py --model resnet56 --momentum 0.1 --batch_size 1024 --save_epoch 1

# Visualizations

## Creating 1D linear interpolations

Currently only for CIFAR10, since we always need 2 models and we don't want to spend any more time training models for the other data sets.

There are also no random directions, since we interpolated between to optima, so we can't do any randomization.

In [ ]:
# CIFAR10
!python plot_surface.py --mpi $CUDA_FLAG --model vgg9 --x=-0.5:1.5:101 --dir_type states \
--model_file cifar10/trained_nets/vgg9_sgd_lr=0.1_bs=128_wd=0.0_save_epoch=1/model_300.t7 \
--model_file2 cifar10/trained_nets/vgg9_sgd_lr=0.1_bs=8192_wd=0.0_save_epoch=1/model_300.t7\
--plot --timestamp --fileformat png

Using manual seed: 123
The current seed is 123.
Rank 0 use GPU 0 of 1 GPUs on 42b14f9786dd
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
cifar10/trained_nets/vgg9_sgd_lr=0.1_bs=128_wd=0.0_save_epoch=1_model_300.t7_vgg9_sgd_lr=0.1_bs=8192_wd=0.0_save_epoch=1_model_300.t7_states.h5 is already set up
cifar10/trained_nets/vgg9_sgd_lr=0.1_bs=128_wd=0.0_save_epoch=1_model_300.t7_vgg9_sgd_lr=0.1_bs=8192_wd=0.0_save_epoch=1_model_300.t7_states.h5_[-0.5,1.5,101].h5 is already set up
Files already downloaded and verified
Files already downloaded and verified
Computing 100 values for rank 0
Traceback (most recent call last):
  File "/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/plot_surface.py", line 327, in <module>
    crunch(surf_file, net, w, s, d, trainloader, 'train_loss', 'train_acc', args)
  File "/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/plot_surface.py", li

## Producing plots along random normalized directions

In [ ]:
# CIFAR10
!python plot_surface.py --mpi $CUDA_FLAG --dataset cifar10 --datapath cifar10/data \
--model resnet56 --x=-1:1:51 \
--model_file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7 \
--dir_type weights --xnorm filter --xignore biasbn\
--plot --timestamp --fileformat png

Using manual seed: 123
The current seed is 123.
Rank 0 use GPU 0 of 1 GPUs on 42b14f9786dd
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter.h5 is already set up
cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter.h5_[-1.0,1.0,51].h5 is already set up
Files already downloaded and verified
Files already downloaded and verified
Computing 0 values for rank 0
Rank 0 done!  Total time: 0.00 Sync: 0.00
Computing 0 values for rank 0
Rank 0 done!  Total time: 0.00 Sync: 0.00
------------------------------------------------------------------
plot_1d_loss_err
------------------------------------------------------------------
<KeysViewHDF5 ['test_acc', 'test_loss', 'train_acc', 'train_loss', 'xcoordinates']>
train_loss
[4.26416056e+00 4.

In [ ]:
# CIFAR10
# last seed used: 18129200292988078306
!python plot_surface.py --mpi $CUDA_FLAG --dataset cifar10 --datapath cifar10/data \
--model resnet56 --x=-1:1:51 \
--model_file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7 \
--dir_type weights --xnorm filter --xignore biasbn \
--random_seed \
--plot --timestamp --fileformat png

Setting the seed randomly.
The current seed is 18129200292988078306.
Rank 0 use GPU 0 of 1 GPUs on 42b14f9786dd
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
Setting up the plotting directions...
direction file created: cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter.h5
Files already downloaded and verified
Files already downloaded and verified
Computing 51 values for rank 0
Evaluating rank 0  0/51  (0.0%)  coord=-1.0 	train_loss= 4.321 	train_acc=12.61 	time=17.28 	sync=0.00
Writing to file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter.h5_[-1.0,1.0,51].h5
Evaluating rank 0  1/51  (2.0%)  coord=-0.96 	train_loss= 4.320 	train_acc=14.87 	time=16.41 	sync=0.00
Evaluating rank 0  2/51  (3.9%)  coord=-0.92 	train_loss= 4.269 	train_acc=17.15 	time=16.43 	sync=0.00
Eva

In [ ]:
# MNIST
# seed used: 12004702179821601059
# seed used: 6687871989849813559
!python plot_surface.py --mpi $CUDA_FLAG --dataset MNIST --datapath MNIST/data \
--model resnet56 --x=-1:1:51 \
--model_file MNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_41.t7 \
--dir_type weights --xnorm filter --xignore biasbn \
--random_seed \
--plot --timestamp --fileformat png

Setting the seed randomly.
The current seed is 6687871989849813559.
Rank 0 use GPU 0 of 1 GPUs on 42b14f9786dd
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
Setting up the plotting directions...
direction file created: MNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_41.t7_weights_xignore=biasbn_xnorm=filter.h5
Computing 51 values for rank 0
Evaluating rank 0  0/51  (0.0%)  coord=-1.0 	train_loss= 0.728 	train_acc=74.82 	time=19.31 	sync=0.00
Writing to file MNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_41.t7_weights_xignore=biasbn_xnorm=filter.h5_[-1.0,1.0,51].h5
Evaluating rank 0  1/51  (2.0%)  coord=-0.96 	train_loss= 0.580 	train_acc=79.98 	time=20.83 	sync=0.00
Evaluating rank 0  2/51  (3.9%)  coord=-0.92 	train_loss= 0.463 	train_acc=84.14 	time=19.71 	sync=0.00
Evaluating rank 0  3/51  (5.9%)  coord=-0.88

In [ ]:
# FashionMNIST
# seed used: 15444593190463038104
!python plot_surface.py --mpi $CUDA_FLAG --dataset FashionMNIST --datapath FashionMNIST/data \
--model resnet56 --x=-1:1:51 \
--model_file FashionMNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_40.t7 \
--dir_type weights --xnorm filter --xignore biasbn \
--random_seed \
--plot --timestamp --fileformat png

Setting the seed randomly.
The current seed is 15444593190463038104.
Rank 0 use GPU 0 of 1 GPUs on 42b14f9786dd
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
Setting up the plotting directions...
direction file created: FashionMNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_40.t7_weights_xignore=biasbn_xnorm=filter.h5
Computing 51 values for rank 0
Evaluating rank 0  0/51  (0.0%)  coord=-1.0 	train_loss= 3.045 	train_acc=32.84 	time=20.50 	sync=0.00
Writing to file FashionMNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_40.t7_weights_xignore=biasbn_xnorm=filter.h5_[-1.0,1.0,51].h5
Evaluating rank 0  1/51  (2.0%)  coord=-0.96 	train_loss= 2.611 	train_acc=36.97 	time=20.02 	sync=0.00
Evaluating rank 0  2/51  (3.9%)  coord=-0.92 	train_loss= 2.257 	train_acc=42.80 	time=19.02 	sync=0.00
Evaluating rank 0  3/51  (5.9

## Visualizing 2D loss contours

call without cuda (sorry need it to copy and paste)


```
python plot_surface.py --mpi --model resnet56 --x=-1:1:5 --y=-1:1:5 \
--model_file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7 \
--dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn  --plot
```


```
python plot_surface.py --mpi --model resnet56 --x=-1:1:5 --y=-1:1:5 \
--xnorm filter --xignore biasbn --ynorm filter --yignore biasbn  --plot
```


```
mpirun -n 4 python plot_surface.py --mpi --model resnet56 --x=-1:1:51 --y=-1:1:51 \
--model_file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7 \
--dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn  --plot
```





In [8]:
# CIFAR10
# seed used: 11639292316745685709

!python plot_surface.py --mpi $CUDA_FLAG --dataset cifar10 --datapath cifar10/data \
--model resnet56 --x=-1:1:21 --y=-1:1:21 \
--model_file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7 \
--dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn\
--plot --timestamp --fileformat png \
--random_seed

Setting the seed randomly.
The current seed is 11639292316745685709.
Rank 0 use GPU 0 of 1 GPUs on f054b91ad7d1
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
Setting up the plotting directions...
direction file created: cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5
cosine similarity between x-axis and y-axis: -0.001391
Files already downloaded and verified
Files already downloaded and verified
Computing 441 values for rank 0
Evaluating rank 0  0/441  (0.0%)  coord=[-1. -1.] 	train_loss= 13.889 	train_acc=9.98 	time=19.93 	sync=0.00
Writing to file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,21]x[-1.0,1.0,21].h5
Evaluating rank 0  1/441  (0.2%)  coord=[-0.9 -1. ] 	train_loss= 10.087 	train_

In [12]:
# Cifar10
!python h52vtp.py \
--surf_file cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,21]x[-1.0,1.0,21].h5 \
--surf_name train_loss --zmax 10 --log

/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:115: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 0:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:118: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:122: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (vertexcount % 6) is not 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:132: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 0:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:135: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:139: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (vertexcount % 6) is not 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper

In [9]:
# MNIST
# seed used: 18079330205315205622

!python plot_surface.py --mpi $CUDA_FLAG --dataset MNIST --datapath MNIST/data \
--model resnet56 --x=-1:1:21 --y=-1:1:21 \
--model_file MNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_41.t7 \
--dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn\
--plot --timestamp --fileformat png \
--random_seed

Setting the seed randomly.
The current seed is 12720805039251181950.
Rank 0 use GPU 0 of 1 GPUs on d281dd310d4c
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
MNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_41.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5 is already set up
MNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_41.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,21]x[-1.0,1.0,21].h5 is already set up
cosine similarity between x-axis and y-axis: 0.002580
Computing 0 values for rank 0
Rank 0 done!  Total time: 0.00 Sync: 0.00
Computing 0 values for rank 0
Rank 0 done!  Total time: 0.00 Sync: 0.00
------------------------------------------------------------------
plot_2d_contour
------------------------------------------------------------------
loading s

In [10]:
# MNIST
!python h52vtp.py \
--surf_file MNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_41.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,21]x[-1.0,1.0,21].h5 \
--surf_name train_loss --zmax 10 --log

/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:115: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 0:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:118: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:122: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (vertexcount % 6) is not 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:132: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 0:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:135: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (vertexcount % 6) is 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper_04/code/h52vtp.py:139: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (vertexcount % 6) is not 5:
/content/drive/MyDrive/Studium/WS_2023/MML/Paper

In [ ]:
# FashionMNIST
# seed used: 12084839173271460687

!python plot_surface.py --mpi $CUDA_FLAG --dataset FashionMNIST --datapath FashionMNIST/data \
--model resnet56 --x=-1:1:21 --y=-1:1:21 \
--model_file FashionMNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_40.t7 \
--dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn \
--random_seed \
--plot --timestamp --fileformat png

In [ ]:
# FashionMNIST
!python h52vtp.py \
--surf_file FashionMNIST/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005_mom=0.9_save_epoch=1/model_40.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,21]x[-1.0,1.0,21].h5 \
--surf_name train_loss --zmax 10 --log

## Plotting the eigenvalue ratio heatmaps

Takes insanely long to calculate (weeks, if not months, if not years).

In [ ]:
!python plot_hessian_eigen.py --mpi $CUDA_FLAG --model $MODEL --x=-1:1:5 --y=-1:1:5 \
--model_file $MODEL_FILE \
--dir_type weights --xnorm filter --xignore biasbn --ynorm filter --yignore biasbn\
--plot --fileformat png

Using manual seed: 123
The current seed is 123.
Rank 0 use GPU 0 of 1 GPUs on e656efa94441
-------------------------------------------------------------------
setup_direction
-------------------------------------------------------------------
cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5 is already set up
cifar10/trained_nets/resnet56_sgd_lr=0.1_bs=128_wd=0.0005/model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,5]x[-1.0,1.0,5].h5 is already set up
True
cosine similarity between x-axis and y-axis: 0.003037
Files already downloaded and verified
Files already downloaded and verified
Computing 25 values for rank 0
The number of parameters is: 851504
Rank 0: computing max eigenvalue
   Iter: 1  time: 104.078464
   Iter: 2  time: 108.540136
   Iter: 3  time: 108.432164
   Iter: 4  time: 108.901910
   Iter: 5  time: 108.583978
   Iter: 6  time: 108.356730
   Iter

# Appendix

## Reverse engineering the structure of .h5 files

In [ ]:
FOLDER_PATH = FILE_PATH + "/cifar10/trained_nets" +\
"/resnet56_noshort_sgd_lr=0.1_bs=128_wd=0.0005"

interesting_attributes = ["min_eig", "max_eig"]

for file_name in os.listdir(FOLDER_PATH):
    if file_name.endswith(".h5"):
        print(f"File name: {file_name}")

        with h5py.File(FOLDER_PATH + "/" + file_name, "r") as f:
          for key in f:
            print(f"Key: {key}")

            if key in interesting_attributes:
              print("Data:")
              try:
                print(list(f[key]))
              except:
                print("Not printable")

        print("---------------------------------------")


File name: model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5
Key: xdirection
Key: ydirection
---------------------------------------
File name: model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter.h5_[-1.0,1.0,201]x[-1.0,1.0,201].h5
Key: dir_file
Key: test_loss
Key: train_acc
Key: train_loss
Key: xcoordinates
Key: ycoordinates
---------------------------------------
File name: model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter_idx=1.h5
Key: xdirection
Key: ydirection
---------------------------------------
File name: model_300.t7_weights_xignore=biasbn_xnorm=filter_yignore=biasbn_ynorm=filter_idx=1.h5_[-1.0,1.0,51]x[-1.0,1.0,51].h5
Key: dir_file
Key: loss_vals
Key: test_loss
Key: train_acc
Key: train_loss
Key: xcoordinates
Key: ycoordinates
---------------------------------------


## Counting the number of iterations in the Implictly Restarted Lanczos Method

In [ ]:
dimension = 1
base = 2
exponent = 0

current_time = 0
max_time = 1000 * 60 * 1 # 10 minutes in microseconds

rows = []

while current_time <= max_time and exponent <= 24:
  exponent += 1
  dimension *= base

  print(f"Current exponent: {exponent}")
  print(f"Current dimension: {dimension}x{dimension}")

  start = time.perf_counter()
  random_spd_matrix = 4 * scipy.sparse.identity(dimension)
  end = time.perf_counter()

  print(f"Generated a s.p.d. matrix of dimensions {dimension}x{dimension}\n\
          in {end - start} seconds.")

  def mat_vec_prod(vector):
    mat_vec_prod.count += 1
    return random_spd_matrix @ vector

  mat_vec_prod.count = 0
  lin_op = scipy.sparse.linalg.LinearOperator((dimension, dimension),
                                               matvec = mat_vec_prod)

  start = time.perf_counter()
  eigvals, eigvecs = scipy.sparse.linalg.eigsh(lin_op, k = 1, tol = 1e-10)
  end = time.perf_counter()

  duration = end - start

  print(f"Calculated eigenvalues and eigenvectors for a s.p.d. matrix of\n\
         dimensions {dimension}x{dimension} using {mat_vec_prod.count}\n\
         iterations and {duration} seconds.")

  print(f"Maximal eigenvalue is: {eigvals[0]}")

  print("---------------------------------------------------------------------")

  temp_dict = {
      "exponent": exponent,
      "dimension": dimension,
      "duration": duration,
      "iterations": mat_vec_prod.count
  }

  rows.append(temp_dict)

df = pd.DataFrame(rows)

Current exponent: 1
Current dimension: 2x2
Generated a s.p.d. matrix of dimensions 2x2
          in 0.3720882709999387 seconds.
Calculated eigenvalues and eigenvectors for a s.p.d. matrix of
         dimensions 2x2 using 3
         iterations and 0.01876304200004597 seconds.
Maximal eigenvalue is: 4.0
---------------------------------------------------------------------
Current exponent: 2
Current dimension: 4x4
Generated a s.p.d. matrix of dimensions 4x4
          in 0.0003531840000050579 seconds.
Calculated eigenvalues and eigenvectors for a s.p.d. matrix of
         dimensions 4x4 using 5
         iterations and 0.0005081829999653564 seconds.
Maximal eigenvalue is: 4.0
---------------------------------------------------------------------
Current exponent: 3
Current dimension: 8x8
Generated a s.p.d. matrix of dimensions 8x8
          in 0.0002558710000357678 seconds.
Calculated eigenvalues and eigenvectors for a s.p.d. matrix of
         dimensions 8x8 using 9
         iterations and

In [ ]:
df

,exponent,dimension,duration,iterations
0,1,2,0.018763,3
1,2,4,0.000508,5
2,3,8,0.000743,9
3,4,16,0.001090,17
4,5,32,0.002668,21
5,6,64,0.002573,21
6,7,128,0.002771,21
7,8,256,0.003178,21
8,9,512,0.004721,21
9,10,1024,0.002031,21


from matplotlib import pyplot as plt
df['exponent'].plot(kind='hist', bins=20, title='exponent')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['dimension'].plot(kind='hist', bins=20, title='dimension')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['duration'].plot(kind='hist', bins=20, title='duration')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['iterations'].plot(kind='hist', bins=20, title='iterations')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['dimension']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'dimension'}, axis=1)
              .sort_values('dimension', ascending=True))
  xs = counted['dimension']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('dimension', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('dimension')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
df['exponent'].plot(kind='line', figsize=(8, 4), title='exponent')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['dimension'].plot(kind='line', figsize=(8, 4), title='dimension')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['duration'].plot(kind='line', figsize=(8, 4), title='duration')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['iterations'].plot(kind='line', figsize=(8, 4), title='iterations')
plt.gca().spines[['top', 'right']].set_visible(False)